In [ ]:
import numpy as np
import torch
import pandas as pd
from tqdm import tqdm
from transformers import BertModel, BertTokenizer, BertForMaskedLM, pipeline
from sentence_transformers import SentenceTransformer, util
from transformers import logging
logging.set_verbosity_error()

print(torch.cuda.is_available())
print(torch.__version__)

import os
from pathlib import Path
os.chdir(Path(os.environ["ALTHOME"]) / "Master-thesis")
# print(os.environ['ALTHOME'])
# print(os.environ['PIP_CACHE_DIR'])
# print(os.environ['TRANSFORMERS_CACHE'])

In [ ]:
l = ["Ä","ä","Ö","ö","Ü","ü","ß"]
replacements = {key: str(bytes(key, encoding="utf-8"), encoding="latin-1") for key in l}

def replace(s):
    for k,v in replacements.items():
        s = s.replace(v, k)
    return s

with open("data/spacy_easy.txt") as fp:
    text = fp.read()
text = replace(text)
with open("data/fixed_easy.txt", "w") as fp:
    fp.write(text)
with open("data/spacy_normal.txt") as fp:
    text = fp.read()
text = replace(text)
with open("data/fixed_normal.txt", "w") as fp:
    fp.write(text)

In [ ]:
model_name = "dbmdz/bert-base-german-cased"
model_name = "deepset/gbert-base"
pipe  = pipeline(model=model_name, tokenizer=model_name, task="feature-extraction")

In [ ]:
with open("data/fixed_easy.txt") as fp:
    lines = [i.strip() for i in fp.readlines()]
    print(len(lines))
    # output = pipe(lines)
    

with open("data/fixed_normal.txt") as fp:
    lines = [i.strip() for i in fp.readlines()]


In [ ]:
from transformers import BertModel, BertTokenizer
sentence = "Dies ist eine Satz"
model_name = "deepset/gbert-base"
model = BertModel.from_pretrained(model_name)
tokenizer = BertTokenizer.from_pretrained(model_name)
tokens = tokenizer.encode(sentence, return_tensors="pt")
output = model.forward(tokens, output_hidden_states=True)
output_pipe = pipe(sentence)

a = output.hidden_states[-1].detach().numpy() # same as pipeline feature extraction
c = output.last_hidden_state.detach().numpy()
b = np.array(output_pipe)

print((a == b).all())
print((a == c).all())
print(f"{type(a)} - {a.shape}")
print(f"{type(b)} - {b.shape}")
print(f"{type(c)} - {c.shape}")


In [ ]:
def generator(file):
    with open(file) as fp:
        lines = [i.strip() for i in fp.readlines()]
    for i, line in enumerate(lines):
        if i>5:
            break
        yield line


g1 = generator("data/fixed_easy.txt")
g2 = generator("data/fixed_normal.txt")

df = pd.DataFrame(columns=range(768))
l = [i for i in tqdm(range(5))]

df = pd.DataFrame(l)
print(df)
